In [1]:
import datetime
import numpy as np
import talib
import matplotlib.pyplot as plt
import mplfinance as mpf
from math import comb
from dataset import kline_data
from strategies import ma_cross_signal
from dateutil import parser

In [ ]:
fast, slow = talib.STOCH(high, low, close, fastk_period=5, slowk_period=3,
                           slowk_matype=0, slowd_period=3, slowd_matype=0)  # Stochastic

## 1D Timeframe

In [53]:
btc_1d = kline_data(interval='1d')
btc_1d = btc_1d.set_index(btc_1d['open_time'].map(parser.parse))
close = np.array(btc_1d['close'])
high = np.array(btc_1d['high'])
low = np.array(btc_1d['low'])

n_candles = len(close)
n_train = int(0.75 * n_candles)
n_test = n_candles - n_train
train = close[:n_train]
test = close[n_train:]

Last saved data candle close time: 2021-08-12 00:00:00


In [54]:
periods = np.arange(2, 11)
fee = 0.001
# Training
ct = 1
results = {}
n_runs = len(periods) ** 3
print(f'Total runs: {n_runs}')
start = datetime.datetime.now()
for fastk_period in periods:
    for slowk_period in periods:
        for slowd_period in periods:
            cash = 1000
            p = 1  # position size
            print(f'*** {ct} / {n_runs}')
            fast, slow = talib.STOCH(high, low, close, fastk_period=fastk_period, slowk_period=slowk_period,
                           slowk_matype=0, slowd_period=slowd_period, slowd_matype=0)
            buys, sells = ma_cross_signal(fast[:n_train], slow[:n_train])
            buys = buys * train
            sells = sells * train
            orders = np.nan_to_num(buys) - np.nan_to_num(sells)
            orders = orders[orders != 0]
            if orders[0] < 0:
                orders = np.delete(orders, 0)
            if orders[-1] > 0:
                orders = np.delete(orders, -1)                        
            buys = orders[0::2]
            sells = - orders[1::2]
            if len(buys) != len(sells):
                break
            profits = (sells / buys - 1) * (1 - fee)
            for profit in profits:
                cash += profit * p * cash
            results[cash] = (fastk_period, slowk_period, slowd_period)
            ct += 1
            elapsed = datetime.datetime.now() - start
            print(f'Elapsed time: {elapsed}')
            print(f'ETA: {(elapsed.seconds * n_runs / ct - elapsed.seconds) / 60} minutes')
max_cash = max(list(results.keys()))
print(f'Max cash: {max_cash}')
best_settings = results[max_cash]
print(f'Best settins: {best_settings}')

Total runs: 729
*** 1 / 729
Elapsed time: 0:00:00.004505
ETA: 0.0 minutes
*** 2 / 729
Elapsed time: 0:00:00.013505
ETA: 0.0 minutes
*** 3 / 729
Elapsed time: 0:00:00.021528
ETA: 0.0 minutes
*** 4 / 729
Elapsed time: 0:00:00.027329
ETA: 0.0 minutes
*** 5 / 729
Elapsed time: 0:00:00.032338
ETA: 0.0 minutes
*** 6 / 729
Elapsed time: 0:00:00.037303
ETA: 0.0 minutes
*** 7 / 729
Elapsed time: 0:00:00.042410
ETA: 0.0 minutes
*** 8 / 729
Elapsed time: 0:00:00.050409
ETA: 0.0 minutes
*** 9 / 729
Elapsed time: 0:00:00.057411
ETA: 0.0 minutes
*** 10 / 729
Elapsed time: 0:00:00.065411
ETA: 0.0 minutes
*** 11 / 729
Elapsed time: 0:00:00.073933
ETA: 0.0 minutes
*** 12 / 729
Elapsed time: 0:00:00.081918
ETA: 0.0 minutes
*** 13 / 729
Elapsed time: 0:00:00.089149
ETA: 0.0 minutes
*** 14 / 729
Elapsed time: 0:00:00.097224
ETA: 0.0 minutes
*** 15 / 729
Elapsed time: 0:00:00.105222
ETA: 0.0 minutes
*** 16 / 729
Elapsed time: 0:00:00.114226
ETA: 0.0 minutes
*** 17 / 729
Elapsed time: 0:00:00.122232
ETA: 0.

Elapsed time: 0:00:01.124338
ETA: 0.05974842767295597 minutes
*** 159 / 729
Elapsed time: 0:00:01.134354
ETA: 0.05927083333333334 minutes
*** 160 / 729
Elapsed time: 0:00:01.145500
ETA: 0.058799171842650114 minutes
*** 161 / 729
Elapsed time: 0:00:01.150722
ETA: 0.058333333333333334 minutes
*** 162 / 729
Elapsed time: 0:00:01.155742
ETA: 0.057873210633946834 minutes
*** 163 / 729
Elapsed time: 0:00:01.163784
ETA: 0.05741869918699187 minutes
*** 164 / 729
Elapsed time: 0:00:01.171289
ETA: 0.05696969696969697 minutes
*** 165 / 729
Elapsed time: 0:00:01.175477
ETA: 0.05652610441767069 minutes
*** 166 / 729
Elapsed time: 0:00:01.179474
ETA: 0.0560878243512974 minutes
*** 167 / 729
Elapsed time: 0:00:01.184476
ETA: 0.05565476190476191 minutes
*** 168 / 729
Elapsed time: 0:00:01.188742
ETA: 0.055226824457593686 minutes
*** 169 / 729
Elapsed time: 0:00:01.192742
ETA: 0.05480392156862745 minutes
*** 170 / 729
Elapsed time: 0:00:01.200046
ETA: 0.05438596491228071 minutes
*** 171 / 729
Elapsed t

Elapsed time: 0:00:01.893661
ETA: 0.02410514541387025 minutes
*** 298 / 729
Elapsed time: 0:00:01.901660
ETA: 0.023968784838350056 minutes
*** 299 / 729
Elapsed time: 0:00:01.909661
ETA: 0.023833333333333335 minutes
*** 300 / 729
Elapsed time: 0:00:01.915651
ETA: 0.02369878183831672 minutes
*** 301 / 729
Elapsed time: 0:00:01.921223
ETA: 0.02356512141280353 minutes
*** 302 / 729
Elapsed time: 0:00:01.927044
ETA: 0.02343234323432343 minutes
*** 303 / 729
Elapsed time: 0:00:01.930046
ETA: 0.02330043859649123 minutes
*** 304 / 729
Elapsed time: 0:00:01.934089
ETA: 0.023169398907103827 minutes
*** 305 / 729
Elapsed time: 0:00:01.938564
ETA: 0.023039215686274508 minutes
*** 306 / 729
Elapsed time: 0:00:01.942564
ETA: 0.022909880564603694 minutes
*** 307 / 729
Elapsed time: 0:00:01.946564
ETA: 0.022781385281385284 minutes
*** 308 / 729
Elapsed time: 0:00:01.951565
ETA: 0.022653721682847894 minutes
*** 309 / 729
Elapsed time: 0:00:01.954565
ETA: 0.022526881720430107 minutes
*** 310 / 729
Elap

Elapsed time: 0:00:02.647748
ETA: 0.022916666666666665 minutes
*** 432 / 729
Elapsed time: 0:00:02.650849
ETA: 0.02278675904541955 minutes
*** 433 / 729
Elapsed time: 0:00:02.658504
ETA: 0.022657450076804916 minutes
*** 434 / 729
Elapsed time: 0:00:02.662504
ETA: 0.022528735632183907 minutes
*** 435 / 729
Elapsed time: 0:00:02.667577
ETA: 0.022400611620795104 minutes
*** 436 / 729
Elapsed time: 0:00:02.672805
ETA: 0.022273073989321133 minutes
*** 437 / 729
Elapsed time: 0:00:02.679808
ETA: 0.022146118721461185 minutes
*** 438 / 729
Elapsed time: 0:00:02.685809
ETA: 0.02201974183750949 minutes
*** 439 / 729
Elapsed time: 0:00:02.691822
ETA: 0.021893939393939393 minutes
*** 440 / 729
Elapsed time: 0:00:02.697806
ETA: 0.0217687074829932 minutes
*** 441 / 729
Elapsed time: 0:00:02.700982
ETA: 0.021644042232277522 minutes
*** 442 / 729
Elapsed time: 0:00:02.700982
ETA: 0.02151993980436418 minutes
*** 443 / 729
Elapsed time: 0:00:02.717303
ETA: 0.021396396396396396 minutes
*** 444 / 729
Elap

Elapsed time: 0:00:03.397454
ETA: 0.016393442622950824 minutes
*** 549 / 729
Elapsed time: 0:00:03.401454
ETA: 0.016272727272727276 minutes
*** 550 / 729
Elapsed time: 0:00:03.405454
ETA: 0.016152450090744102 minutes
*** 551 / 729
Elapsed time: 0:00:03.409454
ETA: 0.01603260869565217 minutes
*** 552 / 729
Elapsed time: 0:00:03.413454
ETA: 0.015913200723327307 minutes
*** 553 / 729
Elapsed time: 0:00:03.417022
ETA: 0.015794223826714804 minutes
*** 554 / 729
Elapsed time: 0:00:03.417022
ETA: 0.01567567567567568 minutes
*** 555 / 729
Elapsed time: 0:00:03.417022
ETA: 0.015557553956834535 minutes
*** 556 / 729
Elapsed time: 0:00:03.417022
ETA: 0.015439856373429085 minutes
*** 557 / 729
Elapsed time: 0:00:03.433639
ETA: 0.015322580645161291 minutes
*** 558 / 729
Elapsed time: 0:00:03.439644
ETA: 0.015205724508050088 minutes
*** 559 / 729
Elapsed time: 0:00:03.445741
ETA: 0.015089285714285717 minutes
*** 560 / 729
Elapsed time: 0:00:03.451644
ETA: 0.014973262032085563 minutes
*** 561 / 729
E

Elapsed time: 0:00:03.985017
ETA: 0.0045658682634730574 minutes
*** 668 / 729
Elapsed time: 0:00:03.989016
ETA: 0.004484304932735424 minutes
*** 669 / 729
Elapsed time: 0:00:03.995339
ETA: 0.004402985074626864 minutes
*** 670 / 729
Elapsed time: 0:00:04.002538
ETA: 0.005762543467461493 minutes
*** 671 / 729
Elapsed time: 0:00:04.008543
ETA: 0.005654761904761907 minutes
*** 672 / 729
Elapsed time: 0:00:04.013534
ETA: 0.0055473006438831055 minutes
*** 673 / 729
Elapsed time: 0:00:04.019181
ETA: 0.005440158259149364 minutes
*** 674 / 729
Elapsed time: 0:00:04.026291
ETA: 0.005333333333333338 minutes
*** 675 / 729
Elapsed time: 0:00:04.031914
ETA: 0.005226824457593689 minutes
*** 676 / 729
Elapsed time: 0:00:04.038920
ETA: 0.0051206302314131 minutes
*** 677 / 729
Elapsed time: 0:00:04.046916
ETA: 0.005014749262536874 minutes
*** 678 / 729
Elapsed time: 0:00:04.054503
ETA: 0.004909180166912123 minutes
*** 679 / 729
Elapsed time: 0:00:04.063497
ETA: 0.004803921568627449 minutes
*** 680 / 729

In [58]:
# Testing
cash = 1000
p = 1
fastk_period, slowk_period, slowd_period = best_settings
fast, slow = talib.STOCH(high, low, close, fastk_period=fastk_period, slowk_period=slowk_period,
                           slowk_matype=0, slowd_period=slowd_period, slowd_matype=0)
buys, sells = ma_cross_signal(fast[-n_test:], slow[-n_test:])
buys = buys * test
sells = sells * test
orders = np.nan_to_num(buys) - np.nan_to_num(sells)
orders = orders[orders != 0]
if orders[0] < 0:
    orders = np.delete(orders, 0)
if orders[-1] > 0:
    orders = np.delete(orders, -1)                        
buys = orders[0::2]
sells = - orders[1::2]
profits = (sells / buys - 1) * (1 - fee)
for profit in profits:
    cash += profit * p * cash
print(f'Test results - Final cash: {cash}')
print(f'Number of test days: {n_test}')

Test results - Final cash: 2386.8653521335477
Number of test days: 365


## 4H Timeframe

In [2]:
btc_4h = kline_data(interval='4h')
btc_4h = btc_4h.set_index(btc_4h['open_time'].map(parser.parse))
close = np.array(btc_4h['close'])
high = np.array(btc_4h['high'])
low = np.array(btc_4h['low'])

n_candles = len(close)
n_train = int(0.75 * n_candles)
n_test = n_candles - n_train
train = close[:n_train]
test = close[n_train:]

Last saved data candle close time: 2021-08-12 16:00:00


In [42]:
# Training
periods = np.arange(2, 11)
fee = 0.001
ct = 1
results = {}
n_runs = len(periods) ** 3
print(f'Total runs: {n_runs}')
start = datetime.datetime.now()
for fastk_period in periods:
    for slowk_period in periods:
        for slowd_period in periods:
            cash = 1000
            p = 1  # position size
            print(f'*** {ct} / {n_runs}')
            fast, slow = talib.STOCH(high, low, close, fastk_period=fastk_period, slowk_period=slowk_period,
                           slowk_matype=0, slowd_period=slowd_period, slowd_matype=0)
            buys, sells = ma_cross_signal(fast[:n_train], slow[:n_train])
            buys = buys * train
            sells = sells * train
            orders = np.nan_to_num(buys) - np.nan_to_num(sells)
            orders = orders[orders != 0]
            if orders[0] < 0:
                orders = np.delete(orders, 0)
            if orders[-1] > 0:
                orders = np.delete(orders, -1)                        
            buys = orders[0::2]
            sells = - orders[1::2]
            if len(buys) != len(sells):
                break
            profits = (sells / buys - 1) * (1 - fee)
            for profit in profits:
                cash += profit * p * cash
            results[cash] = (fastk_period, slowk_period, slowd_period)
            ct += 1
            elapsed = datetime.datetime.now() - start
            print(f'Elapsed time: {elapsed}')
            print(f'ETA: {(elapsed.seconds * n_runs / ct - elapsed.seconds) / 60} minutes')
max_cash = max(list(results.keys()))
print(f'Max cash: {max_cash}')
best_settings = results[max_cash]
print(f'Best settins: {best_settings}')

Total runs: 36
*** 1 / 36
Elapsed time: 0:00:00.021294
ETA: 0.0 minutes
*** 2 / 36
Elapsed time: 0:00:00.040042
ETA: 0.0 minutes
*** 3 / 36
Elapsed time: 0:00:00.059938
ETA: 0.0 minutes
*** 4 / 36
Elapsed time: 0:00:00.071197
ETA: 0.0 minutes
*** 5 / 36
Elapsed time: 0:00:00.086821
ETA: 0.0 minutes
*** 6 / 36
Elapsed time: 0:00:00.102525
ETA: 0.0 minutes
*** 7 / 36
Elapsed time: 0:00:00.133895
ETA: 0.0 minutes
*** 8 / 36
Elapsed time: 0:00:00.154673
ETA: 0.0 minutes
*** 9 / 36
Elapsed time: 0:00:00.173672
ETA: 0.0 minutes
*** 10 / 36
Elapsed time: 0:00:00.193016
ETA: 0.0 minutes
*** 11 / 36
Elapsed time: 0:00:00.204134
ETA: 0.0 minutes
*** 12 / 36
Elapsed time: 0:00:00.232135
ETA: 0.0 minutes
*** 13 / 36
Elapsed time: 0:00:00.251331
ETA: 0.0 minutes
*** 14 / 36
Elapsed time: 0:00:00.270086
ETA: 0.0 minutes
*** 15 / 36
Elapsed time: 0:00:00.286583
ETA: 0.0 minutes
*** 16 / 36
Elapsed time: 0:00:00.302208
ETA: 0.0 minutes
*** 17 / 36
Elapsed time: 0:00:00.317833
ETA: 0.0 minutes
*** 18 /

Elapsed time: 0:00:02.479670
ETA: -0.023733333333333332 minutes
*** 125 / 36
Elapsed time: 0:00:02.500406
ETA: -0.02380952380952381 minutes
*** 126 / 36
Elapsed time: 0:00:02.520851
ETA: -0.02388451443569554 minutes
*** 127 / 36
Elapsed time: 0:00:02.540839
ETA: -0.023958333333333335 minutes
*** 128 / 36
Elapsed time: 0:00:02.561096
ETA: -0.024031007751937984 minutes
*** 129 / 36
Elapsed time: 0:00:02.581027
ETA: -0.024102564102564103 minutes
*** 130 / 36
Elapsed time: 0:00:02.602029
ETA: -0.024173027989821884 minutes
*** 131 / 36
Elapsed time: 0:00:02.620806
ETA: -0.024242424242424242 minutes
*** 132 / 36
Elapsed time: 0:00:02.642638
ETA: -0.02431077694235589 minutes
*** 133 / 36
Elapsed time: 0:00:02.661286
ETA: -0.02437810945273632 minutes
*** 134 / 36
Elapsed time: 0:00:02.683335
ETA: -0.024444444444444446 minutes
*** 135 / 36
Elapsed time: 0:00:02.704608
ETA: -0.02450980392156863 minutes
*** 136 / 36
Elapsed time: 0:00:02.724977
ETA: -0.024574209245742093 minutes
*** 137 / 36
Elap

Elapsed time: 0:00:04.796681
ETA: -0.05662482566248257 minutes
*** 239 / 36
Elapsed time: 0:00:04.819115
ETA: -0.056666666666666664 minutes
*** 240 / 36
Elapsed time: 0:00:04.841399
ETA: -0.056708160442600276 minutes
*** 241 / 36
Elapsed time: 0:00:04.860732
ETA: -0.05674931129476584 minutes
*** 242 / 36
Elapsed time: 0:00:04.880727
ETA: -0.056790123456790124 minutes
*** 243 / 36
Elapsed time: 0:00:04.887825
ETA: -0.05683060109289618 minutes
*** 244 / 36
Elapsed time: 0:00:04.920331
ETA: -0.05687074829931973 minutes
*** 245 / 36
Elapsed time: 0:00:04.939868
ETA: -0.056910569105691054 minutes
*** 246 / 36
Elapsed time: 0:00:04.960861
ETA: -0.056950067476383266 minutes
*** 247 / 36
Elapsed time: 0:00:04.982011
ETA: -0.05698924731182796 minutes
*** 248 / 36
Elapsed time: 0:00:05.002190
ETA: -0.071285140562249 minutes
*** 249 / 36
Elapsed time: 0:00:05.023191
ETA: -0.07133333333333333 minutes
*** 250 / 36
Elapsed time: 0:00:05.045005
ETA: -0.07138114209827356 minutes
*** 251 / 36
Elapsed t

Elapsed time: 0:00:07.093061
ETA: -0.1047008547008547 minutes
*** 351 / 36
Elapsed time: 0:00:07.113157
ETA: -0.10473484848484849 minutes
*** 352 / 36
Elapsed time: 0:00:07.135346
ETA: -0.10476864966949953 minutes
*** 353 / 36
Elapsed time: 0:00:07.154653
ETA: -0.10480225988700566 minutes
*** 354 / 36
Elapsed time: 0:00:07.175011
ETA: -0.10483568075117371 minutes
*** 355 / 36
Elapsed time: 0:00:07.196083
ETA: -0.10486891385767791 minutes
*** 356 / 36
Elapsed time: 0:00:07.215501
ETA: -0.10490196078431373 minutes
*** 357 / 36
Elapsed time: 0:00:07.235972
ETA: -0.10493482309124767 minutes
*** 358 / 36
Elapsed time: 0:00:07.255971
ETA: -0.10496750232126277 minutes
*** 359 / 36
Elapsed time: 0:00:07.277972
ETA: -0.105 minutes
*** 360 / 36
Elapsed time: 0:00:07.298971
ETA: -0.10503231763619576 minutes
*** 361 / 36
Elapsed time: 0:00:07.322762
ETA: -0.10506445672191528 minutes
*** 362 / 36
Elapsed time: 0:00:07.347205
ETA: -0.10509641873278236 minutes
*** 363 / 36
Elapsed time: 0:00:07.36850

Elapsed time: 0:00:09.363597
ETA: -0.13831168831168833 minutes
*** 462 / 36
Elapsed time: 0:00:09.397509
ETA: -0.13833693304535638 minutes
*** 463 / 36
Elapsed time: 0:00:09.417519
ETA: -0.13836206896551723 minutes
*** 464 / 36
Elapsed time: 0:00:09.438886
ETA: -0.13838709677419353 minutes
*** 465 / 36
Elapsed time: 0:00:09.453999
ETA: -0.13841201716738197 minutes
*** 466 / 36
Elapsed time: 0:00:09.469697
ETA: -0.13843683083511776 minutes
*** 467 / 36
Elapsed time: 0:00:09.485321
ETA: -0.13846153846153847 minutes
*** 468 / 36
Elapsed time: 0:00:09.516573
ETA: -0.1384861407249467 minutes
*** 469 / 36
Elapsed time: 0:00:09.532407
ETA: -0.13851063829787236 minutes
*** 470 / 36
Elapsed time: 0:00:09.548137
ETA: -0.13853503184713375 minutes
*** 471 / 36
Elapsed time: 0:00:09.579563
ETA: -0.13855932203389829 minutes
*** 472 / 36
Elapsed time: 0:00:09.600789
ETA: -0.13858350951374207 minutes
*** 473 / 36
Elapsed time: 0:00:09.621824
ETA: -0.13860759493670885 minutes
*** 474 / 36
Elapsed time:

Elapsed time: 0:00:11.675561
ETA: -0.1717948717948718 minutes
*** 572 / 36
Elapsed time: 0:00:11.694561
ETA: -0.1718150087260035 minutes
*** 573 / 36
Elapsed time: 0:00:11.716303
ETA: -0.17183507549361207 minutes
*** 574 / 36
Elapsed time: 0:00:11.721562
ETA: -0.17185507246376813 minutes
*** 575 / 36
Elapsed time: 0:00:11.752964
ETA: -0.171875 minutes
*** 576 / 36
Elapsed time: 0:00:11.768839
ETA: -0.171894858463316 minutes
*** 577 / 36
Elapsed time: 0:00:11.797167
ETA: -0.17191464821222607 minutes
*** 578 / 36
Elapsed time: 0:00:11.816438
ETA: -0.17193436960276337 minutes
*** 579 / 36
Elapsed time: 0:00:11.836523
ETA: -0.17195402298850573 minutes
*** 580 / 36
Elapsed time: 0:00:11.857439
ETA: -0.17197360872059667 minutes
*** 581 / 36
Elapsed time: 0:00:11.879656
ETA: -0.17199312714776632 minutes
*** 582 / 36
Elapsed time: 0:00:11.933881
ETA: -0.1720125786163522 minutes
*** 583 / 36
Elapsed time: 0:00:11.956357
ETA: -0.17203196347031963 minutes
*** 584 / 36
Elapsed time: 0:00:11.978022

Elapsed time: 0:00:14.118032
ETA: -0.22108843537414966 minutes
*** 686 / 36
Elapsed time: 0:00:14.133916
ETA: -0.22110625909752546 minutes
*** 687 / 36
Elapsed time: 0:00:14.158734
ETA: -0.22112403100775194 minutes
*** 688 / 36
Elapsed time: 0:00:14.178727
ETA: -0.22114175133043057 minutes
*** 689 / 36
Elapsed time: 0:00:14.187835
ETA: -0.22115942028985508 minutes
*** 690 / 36
Elapsed time: 0:00:14.204011
ETA: -0.22117703810902076 minutes
*** 691 / 36
Elapsed time: 0:00:14.235649
ETA: -0.22119460500963392 minutes
*** 692 / 36
Elapsed time: 0:00:14.251598
ETA: -0.22121212121212122 minutes
*** 693 / 36
Elapsed time: 0:00:14.267229
ETA: -0.22122958693563882 minutes
*** 694 / 36
Elapsed time: 0:00:14.298476
ETA: -0.22124700239808154 minutes
*** 695 / 36
Elapsed time: 0:00:14.320436
ETA: -0.22126436781609196 minutes
*** 696 / 36
Elapsed time: 0:00:14.342436
ETA: -0.22128168340506935 minutes
*** 697 / 36
Elapsed time: 0:00:14.364940
ETA: -0.22129894937917863 minutes
*** 698 / 36
Elapsed time

In [46]:
# Testing
cash = 1000
p = 1
fastk_period, slowk_period, slowd_period = best_settings
fast, slow = talib.STOCH(high, low, close, fastk_period=fastk_period, slowk_period=slowk_period,
                           slowk_matype=0, slowd_period=slowd_period, slowd_matype=0)
buys, sells = ma_cross_signal(fast[-n_test:], slow[-n_test:])
buys = buys * test
sells = sells * test
orders = np.nan_to_num(buys) - np.nan_to_num(sells)
orders = orders[orders != 0]
if orders[0] < 0:
    orders = np.delete(orders, 0)
if orders[-1] > 0:
    orders = np.delete(orders, -1)                        
buys = orders[0::2]
sells = - orders[1::2]
profits = (sells / buys - 1) * (1 - fee)
for profit in profits:
    cash += profit * p * cash
print(f'Test results - Final cash: {cash}')
print(f'Number of test days: {n_test / 6}')

Test results - Final cash: 3284.499885245418
Number of test days: 364.0


## 1H Timeframe

In [61]:
btc_1h = kline_data(interval='1h')
btc_1h = btc_1h.set_index(btc_1h['open_time'].map(parser.parse))
close = np.array(btc_1h['close'])
high = np.array(btc_1h['high'])
low = np.array(btc_1h['low'])

n_candles = len(close)
n_train = int(0.75 * n_candles)
n_test = n_candles - n_train
train = close[:n_train]
test = close[n_train:]

Last saved data candle close time: 2021-08-14 17:00:00


In [62]:
# Training
periods = np.arange(2, 11)
fee = 0.001
ct = 1
results = {}
n_runs = len(periods) ** 3
print(f'Total runs: {n_runs}')
start = datetime.datetime.now()
for fastk_period in periods:
    for slowk_period in periods:
        for slowd_period in periods:
            cash = 1000
            p = 1  # position size
            print(f'*** {ct} / {n_runs}')
            fast, slow = talib.STOCH(high, low, close, fastk_period=fastk_period, slowk_period=slowk_period,
                           slowk_matype=0, slowd_period=slowd_period, slowd_matype=0)
            buys, sells = ma_cross_signal(fast[:n_train], slow[:n_train])
            buys = buys * train
            sells = sells * train
            orders = np.nan_to_num(buys) - np.nan_to_num(sells)
            orders = orders[orders != 0]
            if orders[0] < 0:
                orders = np.delete(orders, 0)
            if orders[-1] > 0:
                orders = np.delete(orders, -1)                        
            buys = orders[0::2]
            sells = - orders[1::2]
            if len(buys) != len(sells):
                break
            print(len(buys), len(sells))
            profits = (sells / buys - 1) * (1 - fee)
            for profit in profits:
                cash += profit * p * cash
            results[cash] = (fastk_period, slowk_period, slowd_period)
            ct += 1
            elapsed = datetime.datetime.now() - start
            print(f'Elapsed time: {elapsed}')
            print(f'ETA: {(elapsed.seconds * n_runs / ct - elapsed.seconds) / 60} minutes')
max_cash = max(list(results.keys()))
print(f'Max cash: {max_cash}')
best_settings = results[max_cash]
print(f'Best settins: {best_settings}')

Total runs: 729
*** 1 / 729
6168 6168
Elapsed time: 0:00:00.128858
ETA: 0.0 minutes
*** 2 / 729
5509 5509
Elapsed time: 0:00:00.257975
ETA: 0.0 minutes
*** 3 / 729
4843 4843
Elapsed time: 0:00:00.414100
ETA: 0.0 minutes
*** 4 / 729
4559 4559
Elapsed time: 0:00:00.539114
ETA: 0.0 minutes
*** 5 / 729
4436 4436
Elapsed time: 0:00:00.707336
ETA: 0.0 minutes
*** 6 / 729
4397 4397
Elapsed time: 0:00:00.899607
ETA: 0.0 minutes
*** 7 / 729
4327 4327
Elapsed time: 0:00:01.109351
ETA: 1.5020833333333334 minutes
*** 8 / 729
4309 4309
Elapsed time: 0:00:01.293320
ETA: 1.3333333333333333 minutes
*** 9 / 729
4288 4288
Elapsed time: 0:00:01.464934
ETA: 1.1983333333333335 minutes
*** 10 / 729
5455 5455
Elapsed time: 0:00:01.603084
ETA: 1.0878787878787877 minutes
*** 11 / 729
4423 4423
Elapsed time: 0:00:01.741462
ETA: 0.9958333333333333 minutes
*** 12 / 729
4131 4131
Elapsed time: 0:00:01.880749
ETA: 0.917948717948718 minutes
*** 13 / 729
3771 3771
Elapsed time: 0:00:02.014398
ETA: 1.7023809523809523 

4911 4911
Elapsed time: 0:00:13.764561
ETA: 1.3471947194719471 minutes
*** 101 / 729
3772 3772
Elapsed time: 0:00:13.889618
ETA: 1.3318627450980391 minutes
*** 102 / 729
3153 3153
Elapsed time: 0:00:14.021290
ETA: 1.4181229773462785 minutes
*** 103 / 729
2720 2720
Elapsed time: 0:00:14.115286
ETA: 1.4022435897435899 minutes
*** 104 / 729
2441 2441
Elapsed time: 0:00:14.238410
ETA: 1.3866666666666667 minutes
*** 105 / 729
2247 2247
Elapsed time: 0:00:14.350046
ETA: 1.371383647798742 minutes
*** 106 / 729
2142 2142
Elapsed time: 0:00:14.459489
ETA: 1.3563862928348909 minutes
*** 107 / 729
2113 2113
Elapsed time: 0:00:14.611728
ETA: 1.3416666666666666 minutes
*** 108 / 729
2058 2058
Elapsed time: 0:00:14.727289
ETA: 1.3272171253822629 minutes
*** 109 / 729
4926 4926
Elapsed time: 0:00:14.852588
ETA: 1.313030303030303 minutes
*** 110 / 729
3768 3768
Elapsed time: 0:00:14.955222
ETA: 1.2990990990990992 minutes
*** 111 / 729
3190 3190
Elapsed time: 0:00:15.091756
ETA: 1.3772321428571428 minu

4319 4319
Elapsed time: 0:00:25.460311
ETA: 1.1020833333333333 minutes
*** 200 / 729
3264 3264
Elapsed time: 0:00:25.569721
ETA: 1.0945273631840797 minutes
*** 201 / 729
2747 2747
Elapsed time: 0:00:25.679081
ETA: 1.087046204620462 minutes
*** 202 / 729
2468 2468
Elapsed time: 0:00:25.787030
ETA: 1.0796387520525452 minutes
*** 203 / 729
2229 2229
Elapsed time: 0:00:25.905347
ETA: 1.0723039215686276 minutes
*** 204 / 729
2047 2047
Elapsed time: 0:00:26.047355
ETA: 1.1076422764227642 minutes
*** 205 / 729
1914 1914
Elapsed time: 0:00:26.156710
ETA: 1.1001618122977346 minutes
*** 206 / 729
1799 1799
Elapsed time: 0:00:26.269499
ETA: 1.0927536231884056 minutes
*** 207 / 729
1729 1729
Elapsed time: 0:00:26.385075
ETA: 1.0854166666666667 minutes
*** 208 / 729
5163 5163
Elapsed time: 0:00:26.523084
ETA: 1.0781499202551834 minutes
*** 209 / 729
4345 4345
Elapsed time: 0:00:26.637004
ETA: 1.0709523809523809 minutes
*** 210 / 729
3635 3635
Elapsed time: 0:00:26.761989
ETA: 1.0638230647709321 min

2575 2575
Elapsed time: 0:00:37.203618
ETA: 0.8918903803131992 minutes
*** 298 / 729
4255 4255
Elapsed time: 0:00:37.300622
ETA: 0.8868450390189521 minutes
*** 299 / 729
3343 3343
Elapsed time: 0:00:37.397549
ETA: 0.8818333333333332 minutes
*** 300 / 729
3032 3032
Elapsed time: 0:00:37.534370
ETA: 0.8768549280177188 minutes
*** 301 / 729
2738 2738
Elapsed time: 0:00:37.671572
ETA: 0.8719094922737308 minutes
*** 302 / 729
2534 2534
Elapsed time: 0:00:37.827426
ETA: 0.8669966996699671 minutes
*** 303 / 729
2422 2422
Elapsed time: 0:00:37.980954
ETA: 0.8621162280701753 minutes
*** 304 / 729
2318 2318
Elapsed time: 0:00:38.112432
ETA: 0.8804371584699453 minutes
*** 305 / 729
2253 2253
Elapsed time: 0:00:38.240437
ETA: 0.8754901960784314 minutes
*** 306 / 729
2200 2200
Elapsed time: 0:00:38.394594
ETA: 0.8705754614549404 minutes
*** 307 / 729
*** 307 / 729
3611 3611
Elapsed time: 0:00:38.679667
ETA: 0.8656926406926407 minutes
*** 308 / 729
2833 2833
Elapsed time: 0:00:38.801595
ETA: 0.86084

1808 1808
Elapsed time: 0:00:49.338321
ETA: 0.690548523206751 minutes
*** 395 / 729
1741 1741
Elapsed time: 0:00:49.484238
ETA: 0.6867424242424243 minutes
*** 396 / 729
1681 1681
Elapsed time: 0:00:49.593613
ETA: 0.6829554995801848 minutes
*** 397 / 729
3274 3274
Elapsed time: 0:00:49.748790
ETA: 0.6791876046901173 minutes
*** 398 / 729
2476 2476
Elapsed time: 0:00:49.893954
ETA: 0.6754385964912281 minutes
*** 399 / 729
2127 2127
Elapsed time: 0:00:50.017533
ETA: 0.6854166666666667 minutes
*** 400 / 729
1925 1925
Elapsed time: 0:00:50.111272
ETA: 0.6816292601828761 minutes
*** 401 / 729
1834 1834
Elapsed time: 0:00:50.226192
ETA: 0.677860696517413 minutes
*** 402 / 729
1764 1764
Elapsed time: 0:00:50.363265
ETA: 0.674110835401158 minutes
*** 403 / 729
1708 1708
Elapsed time: 0:00:50.460398
ETA: 0.6703795379537953 minutes
*** 404 / 729
1639 1639
Elapsed time: 0:00:50.557592
ETA: 0.6666666666666666 minutes
*** 405 / 729
1589 1589
Elapsed time: 0:00:50.670032
ETA: 0.6629720853858785 minut

1605 1605
Elapsed time: 0:01:01.052093
ETA: 0.48668018931710627 minutes
*** 493 / 729
1551 1551
Elapsed time: 0:01:01.169135
ETA: 0.48363697705802977 minutes
*** 494 / 729
1518 1518
Elapsed time: 0:01:01.272063
ETA: 0.4806060606060607 minutes
*** 495 / 729
1467 1467
Elapsed time: 0:01:01.367674
ETA: 0.47758736559139786 minutes
*** 496 / 729
3011 3011
Elapsed time: 0:01:01.490787
ETA: 0.4745808182427901 minutes
*** 497 / 729
2265 2265
Elapsed time: 0:01:01.570055
ETA: 0.4715863453815262 minutes
*** 498 / 729
1914 1914
Elapsed time: 0:01:01.679727
ETA: 0.4686038744154978 minutes
*** 499 / 729
1742 1742
Elapsed time: 0:01:01.803099
ETA: 0.4656333333333334 minutes
*** 500 / 729
1617 1617
Elapsed time: 0:01:01.962777
ETA: 0.46267465069860286 minutes
*** 501 / 729
1520 1520
Elapsed time: 0:01:02.058709
ETA: 0.4672642762284196 minutes
*** 502 / 729
1453 1453
Elapsed time: 0:01:02.185448
ETA: 0.4642809807819747 minutes
*** 503 / 729
1418 1418
Elapsed time: 0:01:02.320520
ETA: 0.461309523809523

1647 1647
Elapsed time: 0:01:12.512124
ETA: 0.28271186440677976 minutes
*** 590 / 729
1533 1533
Elapsed time: 0:01:12.648848
ETA: 0.28020304568527926 minutes
*** 591 / 729
1452 1452
Elapsed time: 0:01:12.734309
ETA: 0.2777027027027027 minutes
*** 592 / 729
1390 1390
Elapsed time: 0:01:12.843384
ETA: 0.2752107925801011 minutes
*** 593 / 729
1349 1349
Elapsed time: 0:01:12.953294
ETA: 0.27272727272727265 minutes
*** 594 / 729
1303 1303
Elapsed time: 0:01:13.098850
ETA: 0.27400560224089643 minutes
*** 595 / 729
5073 5073
Elapsed time: 0:01:13.201797
ETA: 0.2715044742729307 minutes
*** 596 / 729
4221 4221
Elapsed time: 0:01:13.296762
ETA: 0.2690117252931323 minutes
*** 597 / 729
3522 3522
Elapsed time: 0:01:13.426874
ETA: 0.26652731326644363 minutes
*** 598 / 729
3074 3074
Elapsed time: 0:01:13.521766
ETA: 0.2640511964385086 minutes
*** 599 / 729
2765 2765
Elapsed time: 0:01:13.639665
ETA: 0.2615833333333332 minutes
*** 600 / 729
2551 2551
Elapsed time: 0:01:13.782272
ETA: 0.25912368275097

In [63]:
# Testing
cash = 1000
p = 1
fastk_period, slowk_period, slowd_period = best_settings
fast, slow = talib.STOCH(high, low, close, fastk_period=fastk_period, slowk_period=slowk_period,
                           slowk_matype=0, slowd_period=slowd_period, slowd_matype=0)
buys, sells = ma_cross_signal(fast[-n_test:], slow[-n_test:])
buys = buys * test
sells = sells * test
orders = np.nan_to_num(buys) - np.nan_to_num(sells)
orders = orders[orders != 0]
if orders[0] < 0:
    orders = np.delete(orders, 0)
if orders[-1] > 0:
    orders = np.delete(orders, -1)                        
buys = orders[0::2]
sells = - orders[1::2]
profits = (sells / buys - 1) * (1 - fee)
for profit in profits:
    cash += profit * p * cash
print(f'Test results - Final cash: {cash}')
print(f'Number of test days: {n_test / 24}')

Test results - Final cash: 1725.1749193246355
Number of test days: 363.375
